In [1]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.session import SparkSession
from splink import Splink
from splink.jar_fallback import jw_sim_py




In [2]:

print("PySpark Application Started")

spark = SparkSession \
    .builder \
    .appName("Splink") \
    .master("local[*]") \
    .getOrCreate()

print(spark.sparkContext.appName)

df = spark.read.option("header", "true").csv("/home/abhishekks446/Downloads/Financial-Data3decimal.csv").withColumnRenamed("Entry_id","unique_id")

PySpark Application Started


22/10/12 19:12:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Splink


In [5]:
df=df.na.fill("").limit(1000)
df.show()

+---------+---+------------+----------+------+----------------+----------+------------+------------+------------+------------+-----------------+-------------------+--------------------+--------+
|unique_id|age|pay_schedule|home_owner|income|amount_requested|risk_score|risk_score_2|risk_score_3|risk_score_4|risk_score_5|ext_quality_score|ext_quality_score_2|inquiries_last_month|e_signed|
+---------+---+------------+----------+------+----------------+----------+------------+------------+------------+------------+-----------------+-------------------+--------------------+--------+
|  7629673| 40|   bi-weekly|         1|  3135|             550|     36200| 0.737398319| 0.903517238|   0.4877125| 0.515976695|         0.580918|           0.380918|                  10|       1|
|  3560428| 61|      weekly|         0|  3180|             600|     30150| 0.738510084| 0.881026665| 0.713423437| 0.826401956|          0.73072|            0.63072|                   9|       0|
|  6934997| 23|      week

In [6]:
df.count()


1000

In [20]:
settings = {
    "link_type": "dedupe_only",
    "blocking_rules": [
        "l.pay_schedule = r.pay_schedule "

    ],
    "comparison_columns": [

        {
            "col_name": "pay_schedule"
        },
        {
            "col_name": "risk_score_2",
            "num_levels": 4
        }
    ]
}

linker = Splink(settings,df, spark)
df_e = linker.get_scored_comparisons()

In [21]:
df_1=df_e.toPandas().sort_values("match_probability", ascending=False)

In [22]:
df_1.count()

match_weight          214062
match_probability     214062
unique_id_l           214062
unique_id_r           214062
pay_schedule_l        214062
pay_schedule_r        214062
gamma_pay_schedule    214062
risk_score_2_l        214062
risk_score_2_r        214062
gamma_risk_score_2    214062
dtype: int64

In [23]:
df_1.head(20)

,match_weight,match_probability,unique_id_l,unique_id_r,pay_schedule_l,pay_schedule_r,gamma_pay_schedule,risk_score_2_l,risk_score_2_r,gamma_risk_score_2
29272,4.754888,0.964286,3231870,9742389,bi-weekly,bi-weekly,1,0.688936975,0.688936975,3
71495,1.947533,0.794118,1906268,3601378,bi-weekly,bi-weekly,1,0.724929412,0.528929412,2
18432,1.947533,0.794118,3337988,4694254,bi-weekly,bi-weekly,1,0.624726891,0.623026891,2
41856,1.947533,0.794118,6737150,9972182,bi-weekly,bi-weekly,1,0.768494958,0.76464958,2
161377,1.947533,0.794118,2626256,4838692,bi-weekly,bi-weekly,1,0.795110084,0.735910084,2
98368,1.947533,0.794118,1566038,7254661,semi-monthly,semi-monthly,1,0.633319328,0.631319328,2
39725,1.947533,0.794118,3003105,7658216,bi-weekly,bi-weekly,1,0.629668067,0.679668067,2
103263,1.947533,0.794118,5023159,9438177,bi-weekly,bi-weekly,1,0.580763025,0.785763025,2
106132,1.947533,0.794118,4788815,5935289,bi-weekly,bi-weekly,1,0.736963025,0.838963025,2
107945,1.947533,0.794118,3106804,4733044,bi-weekly,bi-weekly,1,0.626267227,0.612267227,2


In [ ]:
settings = {
    "link_type": "dedupe_only",
    "comparison_columns": [

        {
            "col_name": "title",
            "num_levels": 3,
            "term_frequency_adjustments": True
        },
        {
            "col_name": "year"
        },
        {
            "col_name": "venue"
        }
    ],
    "em_convergence": 0.01,
    "max_iterations": 4,
}

linker = Splink(settings,df, spark)
df_e = linker.get_scored_comparisons()

In [ ]:
df_2=df_e.toPandas().sort_values("match_probability", ascending=False)

In [ ]:
df_2.count()

In [ ]:
df_2.head(10)